In [1]:
import os
import json
import pandas as pd
import traceback

In [2]:
from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv

load_dotenv()


True

In [3]:
KEY = os.getenv("OPEN_API_KEY")

In [52]:
llm = ChatOpenAI(openai_api_key=KEY,model_name='gpt-3.5-turbo',temperature=0.5)

In [54]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
from langchain.chains import LLMChain
import os
import pandas as pd
import traceback
from dotenv import load_dotenv
import PyPDF2

In [56]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [57]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [58]:
quiz_generation_report = PromptTemplate(
     input_variable=['Text', 'number', 'subject', 'tone', 'response_json'],
     template=TEMPLATE
)


In [59]:

quiz_chain = LLMChain(llm=llm, prompt=quiz_generation_report, output_key='quiz', verbose=True)


In [60]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:

"""

In [61]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE)

In [62]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [63]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)

In [64]:
file_path=r"C:\Users\Raviksh\Downloads\openai_env\mcq_generator_project_1\data.txt"

In [65]:
file_path

'C:\\Users\\Raviksh\\Downloads\\openai_env\\mcq_generator_project_1\\data.txt'

In [66]:
with open(file_path,'r') as file:
     TEXT = file.read()

In [67]:
print(TEXT)

Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data and thus perform tasks without explicit instructions.[1] Recently, artificial neural networks have been able to surpass many previous approaches in performance.[2]

ML finds application in many fields, including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine.[3][4] When applied to business problems, it is known under the name predictive analytics. Although not all machine learning is statistically based, computational statistics is an important source of the field's methods.

The mathematical foundations of ML are provided by mathematical optimization (mathematical programming) methods. Data mining is a related (parallel) field of study, focusing on exploratory data analysis (EDA) through unsupervised learning.[6][7]

From a theoretical 

In [68]:
#serializes the python dictionary into a JSON-formatted string
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [69]:
NUMBER=5 
SUBJECT="machine learning"
TONE="simple"

In [70]:
#https://python.langchain.com/docs/modules/model_io/llms/token_usage_tracking

#How to setup Token Usage Tracking in LangChain
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
        )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data and thus perform tasks without explicit instructions.[1] Recently, artificial neural networks have been able to surpass many previous approaches in performance.[2]

ML finds application in many fields, including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine.[3][4] When applied to business problems, it is known under the name predictive analytics. Although not all machine learning is statistically based, computational statistics is an important source of the field's methods.

The mathematical foundations of ML are provided by mathematical optimization (mathematical programming) methods. Data mining is a related (parallel) field of 

In [71]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:1565
Prompt Tokens:894
Completion Tokens:671
Total Cost:0.002683


In [72]:
response

{'text': "Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data and thus perform tasks without explicit instructions.[1] Recently, artificial neural networks have been able to surpass many previous approaches in performance.[2]\n\nML finds application in many fields, including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine.[3][4] When applied to business problems, it is known under the name predictive analytics. Although not all machine learning is statistically based, computational statistics is an important source of the field's methods.\n\nThe mathematical foundations of ML are provided by mathematical optimization (mathematical programming) methods. Data mining is a related (parallel) field of study, focusing on exploratory data analysis (EDA) through unsupervised learning.[6][7]\n\nFro

In [79]:
quiz = response.get("quiz")

In [80]:
quiz=json.loads(quiz)

In [81]:
quiz

{'1': {'mcq': 'What is machine learning?',
  'options': {'a': 'A field of study in artificial intelligence focused on developing algorithms that can learn from data',
   'b': 'A field of study in robotics focused on developing algorithms that can learn from data',
   'c': 'A field of study in biology focused on developing algorithms that can learn from data',
   'd': 'A field of study in physics focused on developing algorithms that can learn from data'},
  'correct': 'a'},
 '2': {'mcq': 'Which field does machine learning find applications in?',
  'options': {'a': 'Natural language processing',
   'b': 'History',
   'c': 'Music',
   'd': 'Geography'},
  'correct': 'a'},
 '3': {'mcq': 'What is the mathematical foundation of machine learning?',
  'options': {'a': 'Algebra',
   'b': 'Geometry',
   'c': 'Mathematical optimization',
   'd': 'Trigonometry'},
  'correct': 'c'},
 '4': {'mcq': 'What is data mining focused on?',
  'options': {'a': 'Supervised learning',
   'b': 'Unsupervised lea

In [82]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [83]:
quiz_table_data

[{'MCQ': 'What is machine learning?',
  'Choices': 'a: A field of study in artificial intelligence focused on developing algorithms that can learn from data | b: A field of study in robotics focused on developing algorithms that can learn from data | c: A field of study in biology focused on developing algorithms that can learn from data | d: A field of study in physics focused on developing algorithms that can learn from data',
  'Correct': 'a'},
 {'MCQ': 'Which field does machine learning find applications in?',
  'Choices': 'a: Natural language processing | b: History | c: Music | d: Geography',
  'Correct': 'a'},
 {'MCQ': 'What is the mathematical foundation of machine learning?',
  'Choices': 'a: Algebra | b: Geometry | c: Mathematical optimization | d: Trigonometry',
  'Correct': 'c'},
 {'MCQ': 'What is data mining focused on?',
  'Choices': 'a: Supervised learning | b: Unsupervised learning | c: Reinforcement learning | d: Deep learning',
  'Correct': 'b'},
 {'MCQ': 'What framew

In [85]:
quiz = pd.DataFrame(quiz_table_data)

In [86]:
quiz.to_csv("machine_learning.csv",index=False)